Projeto Final:\
\
Guia para Legend of Zelda: Breath of the Wild

In [8]:
from plyer import notification
from datetime import datetime
import pandas as pd
import requests
import sqlite3

In [9]:
def alerta(nivel, base, etapa):

    hoje = datetime.now()
    hoje = hoje.strftime("%d/%m/%Y %H:%M:%S")

    message = f'Falha no carregamento da base {base} na etapa {etapa}.\n{hoje}'
    
    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo',
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio',
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto',
    
    notification.notify(
        title = title,
        message = message,
        app_name = 'alerta',
        timeout = 10
        )

Banco de Dados

In [10]:
def retrieve_table():
    conn = sqlite3.connect('zelda.db')
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query,conn)

    conn.close()
    return schema

def save_db(df,table_name):
    conn = sqlite3.connect('zelda.db')
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()
    return True

def load_db(table_name):
    conn = sqlite3.connect('zelda.db')
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query,conn)

    conn.close()
    return df

Extração

In [11]:
url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'

In [12]:
#def get_api(url):
response = requests.get(url)

if response.status_code == 200:
    data_json = response.json()
else:
    print(f'Erro {response.status_code}')

In [13]:
data_json = data_json['data']

In [14]:
#def get_botw():
#url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
#data_json = data_json['data']
zelda_df = pd.DataFrame(data_json)
zelda_df.head(2)

,category,common_locations,description,dlc,drops,id,image,name,cooking_effect,hearts_recovered,properties,edible
0,treasure,[Greater Hyrule],This deposit contains a good deal of precious ...,False,"[ruby, sapphire, diamond, amber, topaz, flint]",388,https://botw-compendium.herokuapp.com/api/v3/c...,rare ore deposit,NaN,NaN,NaN,NaN
1,treasure,[Greater Hyrule],This deposit contains a good deal of ore. Brea...,False,"[ruby, diamond, amber, sapphire, topaz, opal, ...",387,https://botw-compendium.herokuapp.com/api/v3/c...,ore deposit,NaN,NaN,NaN,NaN


In [15]:
name = [name for name in zelda_df['name']]
local = [loc for loc in zelda_df['common_locations']]
category = [cat for cat in zelda_df['category']]
loot = [drop for drop in zelda_df['drops']]

items_df = pd.DataFrame({
    'Nome': name,
    'Região': local,
    'Categoria': category,
    'Loot': loot})
items_df.head(3)

,Nome,Região,Categoria,Loot
0,rare ore deposit,[Greater Hyrule],treasure,"[ruby, sapphire, diamond, amber, topaz, flint]"
1,ore deposit,[Greater Hyrule],treasure,"[ruby, diamond, amber, sapphire, topaz, opal, ..."
2,treasure chest,[Greater Hyrule],treasure,[treasures]


In [16]:
items_reg_exploded = items_df.explode('Região', ignore_index=True)
items_reg_exploded.head(3)

,Nome,Região,Categoria,Loot
0,rare ore deposit,Greater Hyrule,treasure,"[ruby, sapphire, diamond, amber, topaz, flint]"
1,ore deposit,Greater Hyrule,treasure,"[ruby, diamond, amber, sapphire, topaz, opal, ..."
2,treasure chest,Greater Hyrule,treasure,[treasures]


In [17]:
items_loot_exploded = items_reg_exploded.explode('Loot', ignore_index=True)
items_loot = items_loot_exploded.dropna()
items_loot.head(3)

,Nome,Região,Categoria,Loot
0,rare ore deposit,Greater Hyrule,treasure,ruby
1,rare ore deposit,Greater Hyrule,treasure,sapphire
2,rare ore deposit,Greater Hyrule,treasure,diamond


In [18]:
items_df.head(5)

,Nome,Região,Categoria,Loot
0,rare ore deposit,[Greater Hyrule],treasure,"[ruby, sapphire, diamond, amber, topaz, flint]"
1,ore deposit,[Greater Hyrule],treasure,"[ruby, diamond, amber, sapphire, topaz, opal, ..."
2,treasure chest,[Greater Hyrule],treasure,[treasures]
3,luminous ore deposit,[Greater Hyrule],treasure,"[luminous, flint]"
4,dinraal,"[Eldin Mountains, Tabantha Frontier]",monsters,None


In [19]:
effect = [fx for fx in zelda_df['cooking_effect']]
heal = [heal for heal in zelda_df['hearts_recovered']]

effect_df = pd.DataFrame({
    'Nome': name,
    'Efeito': effect,
    'Cura': heal}).dropna()


effect_pivot = effect_df.pivot(index='Nome', columns='Cura', values='Efeito').fillna('')
effect_pivot.head(3)

Cura,0.0,0.5,1.0,2.0,2.5,3.0,4.0,5.0
Nome,,,,,,,,
apple,,,,,,,,
armoranth,defense up,,,,,,,
armored carp,,,defense up,,,,,


In [20]:
effect_df['Efeito'].value_counts()

Efeito
extra hearts        9
                    8
attack up           7
defense up          7
speed up            6
heat resistance     6
stamina recovery    6
stealth up          6
cold resistance     6
shock resistance    6
extra stamina       3
flame guard         2
Name: count, dtype: int64

In [21]:
df_pivot = zelda_df.pivot(index='id', columns='category', values='name')
df_pivot.fillna('').head(3)

category,creatures,equipment,materials,monsters,treasure
id,,,,,
1,horse,,,,
2,giant horse,,,,
3,white horse,,,,


In [22]:
df_pivot.loc[387, 'treasure']

'ore deposit'

In [23]:
df_pivot.iloc[386, 4]

'ore deposit'

In [24]:
df_pivot['creatures'].dropna().head(2)

id
1          horse
2    giant horse
Name: creatures, dtype: object

In [25]:
df_pivot['equipment'].dropna().head(2)

id
201    master sword
202     tree branch
Name: equipment, dtype: object

In [26]:
df_pivot['materials'].dropna().head(2)

id
165         apple
166    palm fruit
Name: materials, dtype: object

In [27]:
df_pivot['monsters'].dropna().head(2)

id
84         chuchu
85    fire chuchu
Name: monsters, dtype: object

In [28]:
df_pivot['treasure'].dropna().head(2)

id
386    treasure chest
387       ore deposit
Name: treasure, dtype: object